In [1]:
# Order of Executing
# If you want to run on a different Query then start from 9

In [2]:
# Order of Executing : 1
from rtree import index
import csv
import pandas as pd
import numpy as np
import timeit
import vptree
import random
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree

In [3]:
# Order of Executing : 2
data = pd.read_csv("HTRU_2.csv",header=None,sep=',') # Loading the data
data = np.array(data) # converting to numpy array
target = data[:,8] # target is the class variable,spambase 57th col is the class variable
data = np.array(data[:,0:8]) # Remaining data

In [4]:
# Order of Executing : 3
data.shape
# pd.DataFrame(data)
# target

(17898, 8)

In [5]:
# Order of Executing : 4 only for the first time
# Creating the index
start = timeit.default_timer()
p = index.Property()
p.dimension = data.shape[1]
idx = index.Index('htru_final', properties=p, interleaved=False) # index name is biotrain_final which is stored in Disk
# idx is the handler
stop = timeit.default_timer()
time_for_nn = stop-start
print("Time for creating R-Tree Index: ",time_for_nn)

Time for creating R-Tree Index:  0.00553810000019439


In [6]:
# Order of Executing : 5
# Assuming that all the points are d dimensional 2-D rectangles
# On each coordinate axis we need two points (x,y) which we assume are the same.
data_list = np.ones((data.shape[0],data.shape[1]*2))
m = 0
for i in range(data.shape[1]):
    for j in range(data.shape[0]):
#         print(m,n,j,i)
#         print(m,n+1,j,i)
        data_list[j,m] = data[j,i]
        data_list[j,m+1] = data[j,i]
    m = m + 2
#         data_list_temp.append(data[i,j])
#     print(data_list_temp)
#     data_list.hstack(data_list_temp)
#     data_list.hstack(data_list_temp)


In [7]:
# Order of Executing : 6
data_list.shape

(17898, 16)

In [8]:
# Order of Executing : 7 
idx

rtree.index.Index(bounds=[1.7976931348623157e+308, -1.7976931348623157e+308, 1.7976931348623157e+308, -1.7976931348623157e+308, 1.7976931348623157e+308, -1.7976931348623157e+308, 1.7976931348623157e+308, -1.7976931348623157e+308, 1.7976931348623157e+308, -1.7976931348623157e+308, 1.7976931348623157e+308, -1.7976931348623157e+308, 1.7976931348623157e+308, -1.7976931348623157e+308, 1.7976931348623157e+308, -1.7976931348623157e+308], size=0)

In [9]:
# Order of Executing : 8 only for the first time
# adding the rectangles to the R-Tree
for i in range(0,data.shape[0]):
    idx.add(i, (data_list[i]))

In [10]:
k = 50

In [11]:
# Order of Executing : 9
# Querying the R-Tree for nearest neighbors
def r_tree_query(query_id,idx):
    start = timeit.default_timer()
    rtree_nn = list(idx.nearest((data_list[query_id]),k))
    stop = timeit.default_timer()
    total_time = stop-start
#     print(time_for_nn)
#     print("Time for R-Tree nearest neighbors:",time_for_nn)
    classes_nn = []
    for i in range(0,k):
        classes_nn.append(int(target[rtree_nn[i]]))
    classes_nn = pd.DataFrame(classes_nn)
    count_classes = dict(classes_nn.groupby(0)[0].count())
    classifier = max(count_classes, key=count_classes.get)
#     print(bool(target[query_id] == classifier))
    return total_time,bool(target[query_id] == classifier)
#     print("The R-Tree classification is: ",bool(target[query_id] == classifier))

In [31]:
# # Order of Executing : 10
# # KD-Tree Construction and recording time
# start = timeit.default_timer()
# kd_tree = KDTree(data)
# stop = timeit.default_timer()
# total_time = stop-start
# print("KD-Tree Construction Time: ",total_time)


KD-Tree Construction Time:  0.01637689999995473


In [32]:
# def kd_tree_query(query_id,kd_tree):
#     j=0
#     # Querying KD-Tree and recording time
#     start = timeit.default_timer()
#     dist_dum , kdtree_nn = kd_tree.query([data[query_id]],10)
#     stop = timeit.default_timer()
#     total_time = stop-start
# #     print(total_time)
# #     print("KD-Tree Query time: ",total_time)
#     # Classifying based on the nearest neigbors found by KD-Tree.
#     kdtree_nn_temp = np.array(kdtree_nn)
#     #Identifying the class of the nearest points.
#     classes_nn = []
#     for i in range(0,10):
#         classes_nn.append(int(target[kdtree_nn_temp[0,i]]))
#     classes_nn = pd.DataFrame(classes_nn)
#     # print(classes_nn)
#     # Finding the count of each class.
#     count_classes = dict(classes_nn.groupby(0)[0].count())
#     # print(count_classes)
#     # len(count_classes)
#     # Identifying the class based on the max count.
#     classifier = max(count_classes, key=count_classes.get)
#     return total_time,bool(target[query_id] == classifier)
#     # print(classifier)
#     # Checking whether the KD-Tree classification with the original classification
# #     print(bool(target[query_id] == classifier))
# #     print("The KD-Tree classification is: ",bool(target[query_id] == classifier))

In [33]:
# # Order of Executing : 11
# # Identifying Original Nearest Neighbors
# original_distances = []
# for i in range(0,data.shape[0]):
#     original_distances.append([i,distance.euclidean(data[10],data[i])]) # Appending [index,distance] to a list
    

# # Sorting the list based on the distance key
# def second_sort(list_ret):
#     return list_ret[1]
# original_distances.sort(key = second_sort) # sorting based on second attribute

# # Classifying based on the top 10
# classes_nn = []
# for i in range(0,10):
#     classes_nn.append(int(target[original_distances[i][0]]))

# classes_nn = pd.DataFrame(classes_nn)

# # print(classes_nn)
# # Finding the count of each class
# count_classes = dict(classes_nn.groupby(0)[0].count())
# classifier = max(count_classes, key=count_classes.get)
# print(classifier)
# print("The Original classification based on 10 neighbours is: ",bool(target[10] == classifier))
# # Classifying based on highest count
# # class_val = count_classes[0]
# # classifier = 0
# # for i in range(0,1): # here 1 is the number of classes in the data starting from 0.
# #     if count_classes[i]> class_val:
# #         classifier = i
# # target[query_id] == classifier # If the target classifier and the tree classifier are same then print true else false.   

# # Getting the data from the main data based on the sorted list first col, which is the index of the data
# # original_result_data = []
# # for i in range(0,11):
# #     original_result_data.append(data_list[original_distances[0][0]]) # Getting only 11 data, coz 10 is k.
# # print(original_result_data)

0
The Original classification based on 10 neighbours is:  True


In [12]:
# Order of Executing : 12 and Only once.
# print(data_list)
# data_list = pd.DataFrame(data_list)
# print(data_list)
# data_list[data_list.shape[1]] = np.arange(len(data_list))
# print(data_list)
# VP-Tree Data Structure
data_list_vp = pd.DataFrame(data) # Converting into dataframe
data_list_vp[data_list_vp.shape[1]] = np.arange(len(data_list_vp)) # Adding a col with index number to identify the nearest point in the data
data_list_vp = np.array(data_list_vp) # COnverting back to array

In [13]:
# Order of Executing : 13
# print(data_list)
# data_list.shape
# data_list.shape
start = timeit.default_timer()
vp_tree = vptree.VPTree(data_list_vp,distance.euclidean) # Building the VP-Tree
stop = timeit.default_timer()
total_time = stop-start
print("Time for Indexing in VP-Tree is: ",total_time)


Time for Indexing in VP-Tree is:  9.242449899998974


In [15]:
def vp_tree_query(query_id,vp_tree):
    start = timeit.default_timer()
    result = vp_tree.get_n_nearest_neighbors(data_list_vp[query_id],k) # Querying the Data Structure
    stop = timeit.default_timer()
    total_time = stop-start
#     print(total_time)

#     print("Time for Querying in VP-Tree is: ",total_time)
    # Seggregating and Gathering the result into a list
    # result_list = []
    # for i in range(0,10):
    #     result_list.append(result[i][1])

    # for i in range(1,10):
    #     print(result[i][1][data.shape[1]-1])

    # print("VP-Tree Accuracy in NN: ",100-(len(np.setdiff1d(original_result_data,result_list))/10)*100)
    # Finding the VP-Tree classification
    # Obtain the results based on the last col of the data
    # print(int(result[1][1][data_list.shape[1]-1]))
    vptree_results = []
    for i in range(1,k):
        vptree_results.append(int(result[i][1][data_list_vp.shape[1]-1]))
    
    # print(vptree_results)
    classes_nn = []
    for i in range(0,len(vptree_results)):
        classes_nn.append(int(target[vptree_results[i]]))
    # print(np.array(classes_nn))

    classes_nn = pd.DataFrame(classes_nn)
    count_classes = dict(classes_nn.groupby(0)[0].count())
    classifier = max(count_classes, key=count_classes.get)
    # print(classifier)
    # Classifying based on highest count
    # class_val = count_classes[0]
    # classifier = 0
    # for i in range(0,1): # here 1 is the number of classes in the data starting from 0.
    #     if count_classes[i]> class_val:
    #         classifier = i
#     print(bool(target[query_id] == classifier))
    return [total_time,bool(target[query_id] == classifier)]
#     print("The VP-Tree classification is: ",bool(target[query_id] == classifier)) 
    # If the target classifier and the tree classifier are same then print true else false.

In [27]:
# kdtree_nn_temp = kdtree_nn
# kdtree_nn_temp = np.array(kdtree_nn)

queries_n = 500
rtee_query_time = []
kd_tree_query_time = []
vp_tree_query_time = []
c = 0
for i in range(queries_n):
    c = c + 1
    query_id = np.random.randint(0,data.shape[0]-1)
    rtee_query_time.append([query_id,r_tree_query(query_id,idx)])
#     kd_tree_query_time.append([query_id,kd_tree_query(query_id,kd_tree)])
    vp_tree_query_time.append([query_id,vp_tree_query(query_id,vp_tree)])




In [28]:
print(c)

500


In [29]:
# Nearest points
# kdtree_nn_temp = np.array(kdtree_nn_temp)
# print(kdtree_nn_temp)
# print(rtee_query_time,kd_tree_query_time,vp_tree_query_time)
# for i in vp_tree_query_time:
# #     print(i[1][0])
#     print(i[0])
avg = 0
true_count = 0
for i in rtee_query_time:
    avg = avg + i[1][0]
    if i[1][1] == True:
        true_count = true_count + 1
print(avg/queries_n)
print((true_count/c)*100)

avg = 0
true_count = 0
for i in vp_tree_query_time:
    avg = avg + i[1][0]
    if i[1][1] == True:
        true_count = true_count + 1
print(avg/queries_n)
print((true_count/c)*100)

# vp_tree_query_time

0.0021351475999908873
98.4
0.01112800219998462
95.39999999999999


In [72]:
# classes_nn = [target[kdtree_nn_temp] for i in range(0,10)]
# # for i in range(0,10):
# #     classes_nn.append(target[kdtree_nn[i]])
# print(classes_nn)
# print(kdtree_nn_temp[0,0])
# Finding the class lables from the data
# print(np.array(classes_nn))
# classes_nn = ['A', 0, 1, 0, 2, 'A', 0, 'A', 3, 'A']
# print(classes_nn)

In [73]:
# classes_dict = {0:0,1:0}
# print(classes_dict)
# for i in range(0,10):
#     classes_dict[classes_nn[i]] = classes_dict[classes_nn[i]]+1
# print(classes_dict)


In [74]:
# # Classifying based on highest count
# class_val = count_classes[0]
# classifier = 0
# for i in range(count_classes.shape[0]): # here 1 is the number of classes in the data starting from 0.
#     if count_classes[i] > class_val:
#         classifier = i


In [75]:
# pd.DataFrame(target).groupby(0)[0].count()

In [76]:
# Find the accuracy of RTree and record the indexing and querying times
# Find the accuracy of KD-Tree and record the indexing and querying times
# Find the accuracy of VP-Tree and record the indexing and querying times


In [77]:
# result
# count = 0
# for i in data_list:
#     if i in result_list:
#         print(count)
#     count +=count
# data_list_vp = list(data_list)
# for i in range(1,10):
#     if result_list[i] in data_list:
#         print(True)

# for i in range(1,10):
#     for j in range(0,data_list.shape[0]):
#         if pd.DataFrame(result_list[i]) == pd.DataFrame(data_list[j]):
#             print(j)


# for i in range(0,len(data_list_vp)):
#     if data_list_vp[i] in result_list:
#         print(i)
# print(type(result_list), type(data_list_vp))